# Добавление истории сообщений

Как правило, пользователь хочет обмениваться сообщениями с вопросно-ответными приложениями.
В таком случае приложению нужна «память» о предыдущих вопросах и ответах и логика для включения этих данных в текущие рассуждения.

В этом разделе показано, как добавить историю сообщений в приложение, разработанное в разделе [Быстрый старт](/docs/use_cases/question_answering/quickstart).

:::note

Подробнее об истории сообщений — в разделе [Работа с историей сообщений](/docs/expression_language/how_to/message_history).
We'll use the following packages:

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai gigachain-chroma bs4

Для добавления истории сообщений в приложение потребуется:

1. Доработать промпт, чтобы он поддерживал историю сообщений на входе.
2. Добавить цепочку, которая берет последний вопрос пользователя и переформулирует его в контексте истории чата. Это нужно, если последний вопрос ссылается на контекст предыдущих сообщений. Например, если пользователь задает уточняющий вопрос: «Расскажи подробнее о втором пункте?». На такой вопрос нельзя ответить без предыдущего контекста.

## Установка зависимостей

Для разработки используются модели генерации и эмбеддингов GigaChat, а также векторное хранилище FAISS.
Вы можете использовать любое [векторное хранилище](/docs/modules/data_connection/vectorstores/) или [ретривер](/docs/modules/data_connection/retrievers/)

Установите пакеты с помощью команды:

In [ ]:
%pip install --upgrade --quiet  gigachain langchainhub chromadb bs4

## Пример цепочки без истории сообщений

Пример цепочки без поддержки истории сообщений, разобранный в разделе [Быстрый старт](/docs/use_cases/question_answering/quickstart):

In [2]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.chat_models import GigaChat
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import GigaChatEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

/Users/18673816/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# Загрузка, разделение на части и индексация содержимого блога.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=GigaChatEmbeddings(
        credentials="<авторизационные_данные>", verify_ssl_certs=False
    ),
)

# Извлечение данных и генерация с помощью релевантных фрагментов блога.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [4]:
rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is a technique used to break down complex tasks into smaller and simpler steps. This approach helps agents to plan and execute tasks more efficiently by dividing them into manageable subgoals. Task decomposition can be achieved through various methods, including using prompting techniques, task-specific instructions, or human inputs.'

## Добавление контекста к вопросу

Сначала нужно определить цепочку, которая:

* получает на вход историю сообщений и последний вопрос пользователя;
* переформулирует вопрос, если он ссылается на данные, доступные в истории сообщений.

Добавим промпт, который содержит переменную `MessagesPlaceholder` под названием `chat_history`.
С помощью ключа `chat_history` в промпт можно передать историю сообщений, которая будет помещена после системного промпта и перед последним вопросом пользователя.

Note that we leverage a helper function create_history_aware_retriever for this step, which manages the case where `chat_history` is empty, and otherwise applies `prompt | llm | StrOutputParser() | retriever` in sequence.

Здесь используется вспомогательная функция [`create_history_aware_retriever`](https://api.python.langchain.com/en/latest/chains/langchain.chains.history_aware_retriever.create_history_aware_retriever.html), которая вызывает цепочку, если список `chat_history` пуст.
Функция принимает на вход ключи и список `chat_history` и возвращает данные, схема которых аналогична схеме выходных данных ретривера.

In [5]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

Полученная цепочка добавляет переформулировку входного запроса к ретриверу, и таким образом позволяет добавить контекст диалога.

## Цепочка с историей чата

Теперь можно собрать итоговую вопросно-ответную логику.

В ней цепочка `contextualize_q_chain` выполняется только при наличии истории сообщений.
Это возможно благодаря тому, что если функция в LCEL-цепочке возвращает другую цепочку, она также будет вызвана.

Для генерации цепочки `question_answer_chain` используется функция [`create_stuff_documents_chain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html).
Для генерации ответа цепочка принимает полученный контекст, наряду с историей чата и запросом пользователя.

Итоговая цепочка `rag_chain` использует `create_retrieval_chain`, которая последовательно вызывает `history_aware_retriever` и `question_answer_chain`.
Таким образом, приложение сохраняет промежуточные результаты, например, извлеченный контекст.
На вход функция примает данные с ключами `input` и `chat_history`, а ее выходные данные содержат ключи `input`, `chat_history`, `context` и `answer`.

In [7]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [8]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Task decomposition can be done in several common ways, including using Language Model (LLM) with simple prompting like "Steps for XYZ" or "What are the subgoals for achieving XYZ?", providing task-specific instructions tailored to the specific task at hand, or incorporating human inputs to guide the decomposition process. These methods help in breaking down complex tasks into smaller, more manageable subtasks for efficient execution.


### Возврат исходных документов

Вопросно-ответным приложениям зачастую важно показывать пользователям исходные данные, которые использовались при генерации.
Для этого вы можете использовать встроенную функцию `create_retrieval_chain`, которая возвращает исходные документы в выходных данных, в поле `context`:

In [9]:
for document in ai_msg_2["context"]:
    print(document)
    print()

page_content='The AI assistant can parse user input to several tasks: [{"task": task, "id", task_id, "dep": dependency_task_ids, "args": {"text": text, "image": URL, "audio": URL, "video": URL}}]. The "dep" field denotes the id of the previous task which generates a new resource that the current task relies on. A special tag "-task_id" refers to the generated text image, audio and video in the dependency task with id as task_id. The task MUST be selected from the following options: {{ Available Task List }}. There is a logical relationship between tasks, please note their order. If the user input can\'t be parsed, you need to reply empty JSON. Here are several cases for your reference: {{ Demonstrations }}. The chat history is recorded as {{ Chat History }}. From this chat history, you can find the path of the user-mentioned resources for your task planning.' metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}

page_content='11. Delete file: "delete_file", args:

## Готовое приложение

![](../../../static/img/conversational_retrieval_chain.png)

Настоящее вопросно-ответное приложение должно уметь сохранять историю сообщений, обновлять ее и использовать в контексте.

Такую функциональность можно добавить с помощью классов:

* [BaseChatMessageHistory](/docs/modules/memory/chat_messages/) — хранение истории сообщений.
* [RunnableWithMessageHistory](/docs/expression_language/how_to/message_history) — обертка для LCEL-цепочки и `BaseChatMessageHistory`, которая преобразует историю сообщений в вводные данные и обновляет ее после каждого вызова.

Подробно о том, как использовать эти классы для разработки разговорной цепочки — в разделе [Работа с историей сообщений](/docs/expression_language/how_to/message_history).

В качестве более простого способа добавление с контекстом, историю сообщений можно хранить в словаре (`dict`).
Полный код такого способа приведен ниже.

In [10]:
import bs4
from langchain import hub
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_models.gigachat import GigaChat
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter

llm = GigaChat(credentials="<авторизационные_данные>", verify_ssl_certs=False)


### Создание ретривера ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=GigaChatEmbeddings(
        credentials="<авторизационные_данные>", verify_ssl_certs=False
    ),
)
retriever = vectorstore.as_retriever()


### Переформулирование вопроса ###
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Ответ на вопрос ###
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Состояние для работы с историей чата ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [11]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. This approach helps agents or models handle difficult tasks by dividing them into more manageable subtasks. It can be achieved through methods like Chain of Thought (CoT) or Tree of Thoughts, which guide the model in thinking step by step or exploring multiple reasoning possibilities at each step.'

In [12]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Task decomposition can be done in common ways such as using Language Model (LLM) with simple prompting, task-specific instructions, or human inputs. For example, LLM can be guided with prompts like "Steps for XYZ" to break down tasks, or specific instructions like "Write a story outline" can be given for task decomposition. Additionally, human inputs can also be utilized to decompose tasks into smaller, more manageable steps.'